In [8]:
# File Settings
INPUT_FILE = "input.mp4"

# this is the output file that will me exported to a "RVE" folder under the "My Drive" folder in Google Drive
OUTPUT_FILE = "output.mkv" # mkv is recommended, as it supports more codecs {mkv, mp4, avi, mov}

# Interpolate Settings

# https://github.com/TNTwise/real-video-enhancer-models/releases/tag/models/ is where all model files for RVE are stored
# you can download them from there, or try custom models from https://openmodeldb.info/
INTERPOLATE = True # {True, False}
INTERPOLATE_FACTOR = 2
SCENE_DETECT_METHOD = "pyscenedetect" # {pyscenedetect, none}
SCENE_DETECT_SENSATIVITY = "3.5" # {0 - 9.9} lower is more sensative

"""
Internal Interpolate Models:
rife4.6.pkl - Speed
rife4.7.pkl - Smoothness
rife4.18.pkl - IRL
rife4.22.pkl - Animation
rife4.25.pkl - General
GMFSS.pkl - Animation
GMFSS_PRO.pkl - Animation
GIMMVFI_RAFT.pth - IRL
"""
INTERPOLATE_MODEL = "rife4.25.pkl" # {Internal model, (link to model path)}


# Upscale Settings
UPSCALE = True # {True, False}
"""
4xNomos8k_span_otf_weak.pth - Realistic 4x High Quality Input
4xNomos8k_span_otf_medium.pth - Realistic 4x Medium Quality Input
4xNomos8k_span_otf_strong.pth - Realistic 4x Low Quality Input
2x_BHI_SpanPlusDynamic_Light.pth - Realistic 2x High Quality Input (pytorch only, experimental)

2x_ModernSpanimationV2.pth - Animation 2x
2x_ModernSpanimationV3.pth - Animation 2x (pytorch only, experimental)
2x_AnimeJaNai_V2_Compact_36k.pth - Animation 2x
2x_AnimeJaNai_HD_V3_Sharp1_Compact_430k.pth - Animation 2x
up2x-latest-conservative.pth - Animation 2x (pytorch only, slow)
up3x-latest-conservative.pth - Animation 3x (pytorch only, slow)
up4x-latest-conservative.pth - Animation 4x (pytorch only, slow)
"""
UPSCALE_MODEL = "2x_ModernSpanimationV2.pth" # {(link to model path)}

# FFMpeg Settings
VIDEO_ENCODER = "libx264" # {libx264,libx265,vp9,av1,prores,ffv1,x264_vulkan,x264_nvenc,x265_nvenc}
AUDIO_ENCODER = "copy_audio" # {aac,libmp3lame,opus,copy_audio}
SUBTITLE_ENCODER = "copy_subtitle" # {srt,ass,webvtt,copy_subtitle}

# Backend Settings
BACKEND = "pytorch" # {pytorch, tensorrt} TensorRT is fastest, but takes a long time to install deps.


In [9]:
# will move output file to drive when done for permident storage
import os
import subprocess
import shutil
from google.colab import drive
# mount drive and move output there for permadent storage

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!rm -rf real-video-enhancer/
!git clone https://github.com/tntwise/real-video-enhancer --branch 2.2.0

Cloning into 'real-video-enhancer'...
remote: Enumerating objects: 23036, done.
remote: Counting objects: 100% (422/422), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 23036 (delta 381), reused 385 (delta 374), pack-reused 22614 (from 2)
Receiving objects: 100% (23036/23036), 10.44 MiB | 15.70 MiB/s, done.
Resolving deltas: 100% (17336/17336), done.


In [11]:

!mkdir real-video-enhancer/models/
origdir = os.getcwd()
os.chdir("real-video-enhancer/models/")


def has_model(model: str):
    return model and model.lower() != "none"

def download_model(model: str):
    os.system(f"wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/{model}")

# get interpolate model by extracting the end of the link
def download_external_model(model_link : str):
        download_model(model_link)
        model = model_link.split(r'/')[-1]
        return model


if INTERPOLATE:
    if "https" in INTERPOLATE_MODEL:
        UPSCALE_MODEL = download_external_model(INTERPOLATE_MODEL, "interpolate")
    else:
        download_model(INTERPOLATE_MODEL)
if UPSCALE:
    if "https" in UPSCALE_MODEL:
        UPSCALE_MODEL = download_external_model(UPSCALE_MODEL, "upscale")
    else:
        download_model(UPSCALE_MODEL)


os.chdir(origdir)



In [12]:
if BACKEND.lower() == 'tensorrt':
    !pip install  --pre --extra-index-url https://download.pytorch.org/whl/test/cu126 -r real-video-enhancer/backend/requirements.txt # only install if using tensorrt

!mkdir bin/ && cd bin && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/ffmpeg-colab/ffmpeg && chmod +x ffmpeg

mkdir: cannot create directory ‘bin/’: File exists


In [13]:
#example upscale/denoise command
# It is recommended to use  --video_encoder_preset x264_nvenc
def build_backend_command() -> str:
    command = ('python3 real-video-enhancer/backend/rve-backend.py'
               + f' -i "{INPUT_FILE}"'
               + f' -o "{OUTPUT_FILE}"'
               + f' -b {BACKEND} '
               + f' --video_encoder_preset {VIDEO_ENCODER}'
               + f' --audio_encoder_preset {AUDIO_ENCODER}'
               + f' --subtitle_encoder_preset {SUBTITLE_ENCODER}')
    if UPSCALE_MODEL and UPSCALE:
        command += f" --upscale_model real-video-enhancer/models/{UPSCALE_MODEL} "
    if INTERPOLATE_MODEL and INTERPOLATE:
        command += f" --interpolate_model real-video-enhancer/models/{INTERPOLATE_MODEL} "
        command += f" --interpolate_factor {INTERPOLATE_FACTOR} "
        command += f" --scene_detect_method {SCENE_DETECT_METHOD} "
        command += f" --scene_detect_threshold {SCENE_DETECT_SENSATIVITY} "

    return command

with open("command.sh", "w") as f:
  f.write(build_backend_command())

!bash command.sh

Using backend: pytorch
Scene Detection Enabled
Using GPU: Tesla T4
Using Interpolation Model: real-video-enhancer/models/rife4.25.pkl
/content/real-video-enhancer/backend/src/pytorch/InterpolateTorch.py:141: UserWarning: The given buffer is not writable, and PyTorch does not support non-writable tensors. This means you can write to the underlying (supposedly non-writable) buffer using the tensor. You may want to copy the buffer to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:1561.)
  torch.frombuffer(
FPS: 7 Current Frame: 4412 ETA: 0:00:00
Time to complete render: 637.81


In [14]:

# Source file path in Colab
source_path = f'/content/{OUTPUT_FILE}'

# Destination path on Google Drive (root directory)
destination_path = f'/content/drive/My Drive/RVE/{OUTPUT_FILE}'
if not os.path.exists(os.path.dirname(destination_path)):
  os.mkdir(os.path.dirname(destination_path))

# Move the file
shutil.move(source_path, destination_path)

'/content/drive/My Drive/RVE/output.mkv'